# Serverless LLMs and Agentic AI with Modal – Lesson 2
## Scaling & Input Concurrency

In this lesson, you’ll explore **how Modal scales your code** and how you can **tune that behavior**.

You will:

- Use a *new* demo function that imitates a small "API-style" workload (not CPU heavy).
- See how Modal automatically scales containers when you submit many tasks.
- Experiment with:
  - `max_containers`, `min_containers`, `scaledown_window`
  - `@modal.concurrent(max_inputs=..., target_inputs=...)`
- Compare behavior **with and without** input concurrency.
- Inspect container activity in the **Modal dashboard**.

> This notebook assumes you already know how to authenticate with Modal from Lesson 1.


In [ ]:
# =====================================
# Step 0 – Install and check Modal
# =====================================
# Run this cell once at the start of your Colab session.

!pip install modal --quiet

# Show where the `modal` CLI is and which version is installed
!which modal
!modal --version

print("✅ Modal installed and detected above (path + version).")

## Step 1 – Make sure you are authenticated

Before you can run this notebook successfully, your environment must be authenticated with Modal.

If you are in **Colab** and didn’t configure a token yet, do this **in a terminal or local shell**:

```bash
modal token set --token-id <YOUR_TOKEN_ID> --token-secret <YOUR_TOKEN_SECRET>
modal token -h
```

If you already did this in a previous lesson on the same machine, you’re good to go.

In this notebook, we’ll just *verify* that Modal can talk to the server.

In [ ]:
!modal token set --token-id <YOUR_TOKEN_ID> --token-secret <YOUR_TOKEN_SECRET>
!modal token -h || echo 'Auth error - run "modal token set" in a terminal and retry.'

## Step 2 – A New Demo Scenario: Simulated “API-style” Requests

We’ll use a **new function** that acts like a small API call:

- It pretends to:
  - receive a request (with an ID and payload size)
  - spend some time “waiting” (simulating network / IO)
  - lightly touch the CPU (e.g., a short checksum-like loop)
- It returns a tiny response dictionary with timing info.

We’ll run this function many times in parallel to see how Modal scales.

You’ll be able to tweak:
- the **sleep time** to simulate slower/faster requests
- the **number of requests** (e.g. 20 vs 500 vs 2000)
- the **scaling and concurrency settings**

In [ ]:
import time
import random
from typing import Dict

import modal

# Create a dedicated app for this lesson
app = modal.App("lesson2-scaling-and-concurrency")


def simulate_api_request(request_id: int, payload_kb: int, base_latency: float = 0.3) -> Dict:
    """Simulate handling a small 'API-style' request.

    This is NOT CPU heavy. It intentionally looks like:
      - A bit of network / IO waiting (time.sleep)
      - A tiny amount of CPU work (a short loop)

    Parameters
    ----------
    request_id : int
        Just an identifier so we can track which call this was.
    payload_kb : int
        A fake "payload size" in kilobytes, used to slightly vary the work.
    base_latency : float
        Minimum time (in seconds) that each request takes.

    Returns
    -------
    Dict with:
      - request_id
      - payload_kb
      - simulated_latency
      - pseudo_checksum
    """
    # Simulate variable latency: base time plus up to 200 ms jitter
    jitter = random.uniform(0.0, 0.2)
    simulated_latency = base_latency + jitter

    # Sleep to mimic waiting on a database / external API
    time.sleep(simulated_latency)

    # Light CPU work: a tiny checksum based on payload size
    # (This is intentionally small so the example stays IO-ish.)
    pseudo_checksum = 0
    for i in range(payload_kb * 10):
        pseudo_checksum = (pseudo_checksum + i) % 97

    return {
        "request_id": request_id,
        "payload_kb": payload_kb,
        "simulated_latency": round(simulated_latency, 3),
        "checksum": pseudo_checksum,
    }

## Step 3 – Auto-scaling configuration for our function

Modal will **auto-scale** by default, but we can tune behavior by adding parameters to `@app.function`:

- `max_containers` – upper limit on containers for this function
- `min_containers` – keep a minimum number of warm containers around
- `scaledown_window` – how long idle containers should stay alive before being shut down

We’ll create a function `handle_request_scaled` where you can play with these settings.

> 🧪 **Experiment:** Run with different values and watch the Modal dashboard.
> For example:
> - `max_containers=2`  vs  `max_containers=20`
> - `min_containers=0`  vs  `min_containers=5`
> - `scaledown_window=None`  vs  `scaledown_window=45`

In [ ]:
# ==================================================
# Function 1 – Focus on container scaling behavior
# ==================================================
#
# Start with everything = None to let Modal decide.
# Then try setting your own values and compare.
#

@app.function(
    max_containers=None,      # e.g. 2, 5, 20
    min_containers=None,      # e.g. 2 or 5
    scaledown_window=None,    # e.g. 30 or 60 seconds
)
def handle_request_scaled(request_id: int, payload_kb: int) -> Dict:
    """Handle a single 'API request' with configurable scaling.

    This function is identical in logic to `simulate_api_request`,
    but runs inside a Modal container and uses scaling settings
    from the decorator above.
    """
    return simulate_api_request(request_id=request_id, payload_kb=payload_kb)

## Step 4 – Input concurrency (many inputs per container)

By default, each Modal container processes **one input at a time**.

For IO-bound workloads like our simulated API call, we might prefer:

> “Fewer containers, each handling many requests in parallel.”

This is where `@modal.concurrent` comes in:

- `max_inputs` – maximum number of in-flight calls per container
- `target_inputs` – soft target concurrency level (autoscaler tries to hover around this)

We’ll make another function `handle_request_concurrent` to explore this.

> ⚠️ Important: `@modal.concurrent` must come **below** `@app.function` in the decorator stack.

In [ ]:
# ==================================================
# Function 2 – Enable input concurrency per container
# ==================================================
#
# Try combinations like:
#   max_inputs=50, target_inputs=40
#   max_inputs=100, target_inputs=80
#
# Start with small values to see the effect more clearly.
#

@app.function()
@modal.concurrent(
    max_inputs=50,   # REQUIRED: try 20, 50, 100, 200
    target_inputs=40 # OPTIONAL: soft target near max_inputs
)
def handle_request_concurrent(request_id: int, payload_kb: int) -> Dict:

    """Handle a single 'API request' with input concurrency enabled.

    With `@modal.concurrent`, a single container can process many requests
    at the same time, which is ideal for IO-bound workloads like this one.
    """
    return simulate_api_request(request_id=request_id, payload_kb=payload_kb, base_latency=0.2)

## Step 5 – Main entrypoint: batch of requests

Our entrypoint will:

1. Generate a list of “requests”
   - request IDs: 0, 1, 2, ..., `num_requests - 1`
   - payload sizes: random small values (e.g. 10–80 KB)
2. Decide whether to use:
   - `handle_request_scaled` (focus on container scaling)
   - `handle_request_concurrent` (focus on input concurrency)
3. Use `.remote(...)` in a loop to submit all requests to Modal.
4. Measure how long everything took.

> For Colab, you typically copy this notebook into a `.py` file and run it with Modal:
>
> ```bash
> modal run lesson2_scaling.py --num-requests 200
> ```

In [ ]:
@app.local_entrypoint()
def main(
    use_concurrency: bool = False,
    num_requests: int = 100,
):
    """Lesson 2 entrypoint.

    Parameters
    ----------
    use_concurrency : bool
        If True, use `handle_request_concurrent` (with input concurrency).
        If False, use `handle_request_scaled` (pure autoscaling focus).
    num_requests : int
        How many simulated API calls to send to Modal.

    Usage examples (from a shell/terminal):
    ---------------------------------------
    - Pure scaling experiment:
        modal run lesson2_scaling.py --num-requests 200

    - Concurrency experiment:
        modal run lesson2_scaling.py --use-concurrency --num-requests 500
    """
    import time as _time
    import random as _random

    print("\n===============================================")
    print("Lesson 2 – Scaling & Input Concurrency in Modal")
    print("===============================================\n")


    fn = handle_request_concurrent if use_concurrency else handle_request_scaled
    print(f"Using function: {fn_label}")
    print(f"Number of simulated API requests: {num_requests}\n")


    # Build a batch of fake requests
    request_ids = list(range(num_requests))
    payload_sizes = [_random.randint(10, 80) for _ in range(num_requests)]  # KB

    inputs = list(zip(request_ids, payload_sizes))
    print("Submitting requests to Modal...\n")


    start = _time.time()

    # Submit each request as a separate remote call
    results = []
    for req_id, size in inputs:
        results.append(fn.remote(req_id, size))

    total_duration = round(_time.time() - start, 2)

    print(f"Completed {len(results)} requests in {total_duration} seconds.")


    # Show a small sample of results
    print("\nSample responses:")
    for r in results[:5]:
        print(r)

    print("\n👉 Now open your Modal dashboard and observe:")
    print("   - How many containers are running at peak load")
    print("   - How quickly containers appear and disappear")
    print("   - Whether input concurrency changes the pattern\n")

    print("🧪 Try changing:")
    print("   - max_containers / min_containers / scaledown_window in `handle_request_scaled`")
    print("   - max_inputs / target_inputs in `handle_request_concurrent`")
    print("   - num_requests and see how behavior shifts.")

In [ ]:
# ==========================================
# Extra cell – export Lesson 2 as lesson2_scaling.py
# ==========================================
# Run this cell once. It will (re)create a Python script
# that you (and students) can run locally with:
#
#   modal run lesson2_scaling.py --num-requests 200
#   modal run lesson2_scaling.py --use-concurrency True --num-requests 500
#
# The script contains:
#   - simulate_api_request
#   - handle_request_scaled  (scaling experiment)
#   - handle_request_concurrent  (input concurrency experiment)
#   - main() as the Modal local entrypoint

%%writefile lesson2_scaling.py
import time
import random
from typing import Dict

import modal

# App for this lesson
app = modal.App("lesson2-scaling-and-concurrency")


def simulate_api_request(request_id: int, payload_kb: int, base_latency: float = 0.3) -> Dict:
    """
    Simulate handling a small 'API-style' request.

    This is NOT CPU heavy. It imitates:
      - Waiting on IO (time.sleep)
      - A tiny amount of CPU work (short loop)

    Parameters
    ----------
    request_id : int
        Just an identifier so we can track which call this was.
    payload_kb : int
        Fake "payload size" in kilobytes, used to slightly vary the work.
    base_latency : float
        Minimum time (in seconds) that each request takes.
    """
    # Simulate variable latency: base time plus up to 200 ms jitter
    jitter = random.uniform(0.0, 0.2)
    simulated_latency = base_latency + jitter

    # Sleep to mimic waiting on a database / external API
    time.sleep(simulated_latency)

    # Light CPU work: tiny checksum based on payload size
    pseudo_checksum = 0
    for i in range(payload_kb * 10):
        pseudo_checksum = (pseudo_checksum + i) % 97

    return {
        "request_id": request_id,
        "payload_kb": payload_kb,
        "simulated_latency": round(simulated_latency, 3),
        "checksum": pseudo_checksum,
    }


# ==================================================
# Function 1 – Focus on container scaling behavior
# ==================================================
#
# Start with everything = None to let Modal decide.
# Then try setting your own values and compare.
#
@app.function(
    max_containers=None,      # e.g. 2, 5, 20
    min_containers=None,      # e.g. 2 or 5
    scaledown_window=None,    # e.g. 30 or 60 seconds
)
def handle_request_scaled(request_id: int, payload_kb: int) -> Dict:
    """Handle a single 'API request' with configurable scaling."""
    return simulate_api_request(request_id=request_id, payload_kb=payload_kb)


# ==================================================
# Function 2 – Enable input concurrency per container
# ==================================================
#
# We start with a concrete default:
#   max_inputs=50 (required)
#   target_inputs=40 (soft target)
#
# Students can change these numbers and re-run.
#
@app.function()
@modal.concurrent(
    max_inputs=50,   # REQUIRED: try 20, 50, 100, 200
    target_inputs=40 # OPTIONAL: soft target near max_inputs
)
def handle_request_concurrent(request_id: int, payload_kb: int) -> Dict:
    """Handle a single 'API request' with input concurrency enabled."""
    return simulate_api_request(request_id=request_id, payload_kb=payload_kb, base_latency=0.2)


@app.local_entrypoint()
def main(
    use_concurrency: bool = False,
    num_requests: int = 100,
):
    """
    Lesson 2 entrypoint.

    Parameters
    ----------
    use_concurrency : bool
        If True, use `handle_request_concurrent` (with input concurrency).
        If False, use `handle_request_scaled` (pure autoscaling focus).
    num_requests : int
        How many simulated API calls to send to Modal.

    Usage examples (from a shell/terminal)
    --------------------------------------
    # Pure scaling experiment:
    modal run lesson2_scaling.py --num-requests 200

    # Concurrency experiment:
    modal run lesson2_scaling.py --use-concurrency --num-requests 500
    """
    import random as _random
    import time as _time

    print("\n===============================================")
    print("Lesson 2 – Scaling & Input Concurrency in Modal")
    print("===============================================\n")

    # Decide which Modal function to use
    if use_concurrency:
        fn = handle_request_concurrent
        fn_label = "handle_request_concurrent (with @modal.concurrent)"
    else:
        fn = handle_request_scaled
        fn_label = "handle_request_scaled (autoscaling only)"

    print(f"Using function: {fn_label}")
    print(f"Number of simulated API requests: {num_requests}\n")

    # Build a batch of fake requests
    request_ids = list(range(num_requests))
    payload_sizes = [_random.randint(10, 80) for _ in range(num_requests)]  # KB
    inputs = list(zip(request_ids, payload_sizes))

    print("Submitting requests to Modal...\n")

    start = _time.time()

    # Submit each request as a separate remote call
    results = []
    for req_id, size in inputs:
        results.append(fn.remote(req_id, size))

    total_duration = round(_time.time() - start, 2)
    print(f"Completed {len(results)} requests in {total_duration} seconds.")

    # Show a small sample of results
    print("\nSample responses:")
    for r in results[:5]:
        print(r)

    print("\n👉 Now open your Modal dashboard and observe:")
    print("   - How many containers are running at peak load")
    print("   - How quickly containers appear and disappear")
    print("   - Whether input concurrency changes the pattern\n")

    print("🧪 Try changing:")
    print("   - max_containers / min_containers / scaledown_window in `handle_request_scaled`")
    print("   - max_inputs / target_inputs in `handle_request_concurrent`")
    print("   - num_requests and see how behavior shifts.")

In [ ]:
!modal run lesson2_scaling.py --num-requests 200

In [ ]:
!modal run lesson2_scaling.py --use-concurrency --num-requests 500